In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../..'))

if module_path not in sys.path:
    sys.path.append(module_path)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from math import sqrt
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

from helper import series_to_supervised
from model.mlp import mlp_layer

2024-02-02 14:31:39.358477: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
# import random

# random.seed(10)
# print(random.random())

In [4]:
dataset = pd.read_csv('../../data/energy_weather.csv', index_col=0)
# https://www.kaggle.com/datasets/nicholasjhana/energy-consumption-generation-prices-and-weather

dataset.fillna(0, inplace=True)
data = dataset
data

,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,pressure_Bilbao,...,wind_deg_Bilbao,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,1035.0,...,229.0,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0
2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,1036.0,...,224.0,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0
2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,1035.0,...,225.0,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0
2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,1035.0,...,221.0,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0
2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,1035.0,...,224.0,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,1033.0,...,57.0,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0
2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,1034.0,...,0.0,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0
2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,1034.0,...,140.0,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0


In [5]:
data.columns

Index(['price', 'price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
      dtype='object')

In [6]:
print(data['load_actual'].min())
print(data['load_actual'].max())

18041.0
41015.0


In [7]:
data.columns

Index(['price', 'price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
      dtype='object')

In [8]:
# for col in data.columns:
#     plt.hist(data[col], bins=20)
#     plt.title(col)
#     plt.show()
#     plt.close()

In [9]:
np.percentile(data['gen_gas'], 95)

10259.0

### Preprocess

In [10]:
values = data.values

# specify the number of lag hours
n_hours = 24*3
n_features = data.shape[-1]
k = 12
split1 = 0.7
split2 = 0.85

# frame as supervised learning
reframed = series_to_supervised(values, n_hours, k)
print("reframed.shape:", reframed.shape)

reframed.shape: (34980, 2184)


In [11]:
# split into train and test sets
reframed_values = reframed.values
n_train_hours = int(len(reframed_values)*split1)
n_valid_hours = int(len(reframed_values)*split2)

train = reframed_values[:n_train_hours, :]
val = reframed_values[n_train_hours:n_valid_hours, :]
test = reframed_values[n_valid_hours:, :]


# split into input and outputs
n_obs = n_hours * n_features
feature_idx = 4
train_X, train_y = train[:, :n_obs], train[:, [n_obs + feature_idx + n_features * i for i in range(k)]]
val_X, val_y = val[:, :n_obs], val[:, [n_obs + feature_idx + n_features * i for i in range(k)]]
test_X, test_y = test[:, :n_obs], test[:, [n_obs + feature_idx + n_features * i for i in range(k)]]


print("train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape", 
      train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape
     )

train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape (24486, 1872) (24486, 12) (5247, 1872) (5247, 12) (5247, 1872) (5247, 12)


In [12]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))

train_X = scaler.fit_transform(train_X)
train_y = scaler.fit_transform(train_y)

val_X = scaler.fit_transform(val_X)
val_y = scaler.fit_transform(val_y)

test_X = scaler.fit_transform(test_X)
test_y = scaler.fit_transform(test_y)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
val_X = val_X.reshape((val_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

print("train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape", 
      train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape
     )

train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape (24486, 72, 26) (24486, 12) (5247, 72, 26) (5247, 12) (5247, 72, 26) (5247, 12)


### Load threshold

In [17]:
train_X_pm = train_X[:, 0, feature_idx]
print(train_X_pm.shape)

val_X_pm = val_X[:, 0, feature_idx]
print(val_X_pm.shape)

test_X_pm = test_X[:, 0, feature_idx]
print(test_X_pm.shape)

(24486,)
(5247,)
(5247,)


In [18]:
percentile = 95

merged_array = np.concatenate((train_X_pm, val_X_pm, test_X_pm))

percentile_pm = np.percentile(merged_array, percentile)

print("{}th Percentile of Daily Rain:".format(percentile), percentile_pm)

95th Percentile of Daily Rain: 0.793594933402977


### train_X_filter

In [19]:
train_X_extreme = train_X[train_X_pm > percentile_pm]
print(train_X_extreme.shape)

train_y_extreme = train_y[train_X_pm > percentile_pm]
print(train_y_extreme.shape)

(976, 72, 26)
(976, 12)


In [20]:
train_X_normal = train_X[train_X_pm <= percentile_pm]
print(train_X_normal.shape)

train_y_normal = train_y[train_X_pm <= percentile_pm]
print(train_y_normal.shape)

(23510, 72, 26)
(23510, 12)


### val_X_filter

In [21]:
val_X_extreme = val_X[val_X_pm > percentile_pm]
print(val_X_extreme.shape)

val_y_extreme = val_y[val_X_pm > percentile_pm]
print(val_y_extreme.shape)

(501, 72, 26)
(501, 12)


In [22]:
val_X_normal = val_X[val_X_pm <= percentile_pm]
print(val_X_normal.shape)

val_y_normal = val_y[val_X_pm <= percentile_pm]
print(val_y_normal.shape)

(4746, 72, 26)
(4746, 12)


### test_X_filter

In [23]:
test_X_extreme = test_X[test_X_pm > percentile_pm]
print(test_X_extreme.shape)

test_y_extreme = test_y[test_X_pm > percentile_pm]
print(test_y_extreme.shape)

(272, 72, 26)
(272, 12)


In [24]:
test_X_normal = test_X[test_X_pm <= percentile_pm]
print(test_X_normal.shape)

test_y_normal = test_y[test_X_pm <= percentile_pm]
print(test_y_normal.shape)

(4975, 72, 26)
(4975, 12)


### Model & training

In [25]:
# plt.hist(train_y[:,0])

In [26]:
# ===== model parameters ======
mlp_unit1 = 128
mlp_unit2 = 128
mlp_unit3 = 64
mlp_unit4 = 64
mlp_unit5 = 32
mlp_unit6 = 32
mlp_unit7 = 16
mlp_unit8 = 16
dropout = 0.0  # 0.1
kernel_size = 2
pool_size = 2
learning_rate = 1e-4
decay_steps = 10000
decay_rate = 0.95
PATIENCE = 100
EPOCHS = 1000
BATCH = 512
opt_num = k
input_shape = train_X.shape[1:]

In [27]:
model = mlp_layer(input_shape=input_shape,
                   mlp_unit1=mlp_unit1,
                   mlp_unit2=mlp_unit2,
                   mlp_unit3=mlp_unit3,
                   mlp_unit4=mlp_unit4,
                   mlp_unit5=mlp_unit5,
                   mlp_unit6=mlp_unit6,
                   mlp_unit7=mlp_unit7,
                   mlp_unit8=mlp_unit8,
                   dropout=dropout,
                   masked_value=-1,
                   opt_num=opt_num
                  )
# model.summary()

In [28]:
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate, 
#                                                           decay_steps=decay_steps,
#                                                           decay_rate=decay_rate)

# model.compile(optimizer=Adam(learning_rate=lr_schedule),
#               loss='mse',
#               metrics=['mae']
#              )

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae']
             )


es = EarlyStopping(monitor='val_mae', mode='min', verbose=2, patience=PATIENCE)
mc = ModelCheckpoint('../../saved_models_mlp/load_E.h5', 
                     monitor='val_mae', 
                     mode='min', 
                     verbose=2, 
                     save_best_only=True
                    )


model.fit(train_X_extreme, train_y_extreme,
          validation_data=(val_X_extreme, val_y_extreme),
          epochs=EPOCHS,
          batch_size=BATCH,
          verbose=2,
          shuffle=True,
          callbacks=[es, mc]
         )

2024-02-02 14:36:19.173358: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-02-02 14:36:19.174237: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1700105000 Hz


Epoch 1/1000


2024-02-02 14:36:20.611937: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


2/2 - 3s - loss: 0.3087 - mae: 0.5183 - val_loss: 0.2795 - val_mae: 0.4869

Epoch 00001: val_mae improved from inf to 0.48686, saving model to ../../saved_models_mlp/load_N.h5
Epoch 2/1000
2/2 - 0s - loss: 0.2796 - mae: 0.4893 - val_loss: 0.2192 - val_mae: 0.4196

Epoch 00002: val_mae improved from 0.48686 to 0.41964, saving model to ../../saved_models_mlp/load_N.h5
Epoch 3/1000
2/2 - 0s - loss: 0.1952 - mae: 0.3918 - val_loss: 0.0975 - val_mae: 0.2549

Epoch 00003: val_mae improved from 0.41964 to 0.25487, saving model to ../../saved_models_mlp/load_N.h5
Epoch 4/1000
2/2 - 0s - loss: 0.0802 - mae: 0.2272 - val_loss: 0.0751 - val_mae: 0.2269

Epoch 00004: val_mae improved from 0.25487 to 0.22685, saving model to ../../saved_models_mlp/load_N.h5
Epoch 5/1000
2/2 - 0s - loss: 0.0751 - mae: 0.2285 - val_loss: 0.0957 - val_mae: 0.2615

Epoch 00005: val_mae did not improve from 0.22685
Epoch 6/1000
2/2 - 0s - loss: 0.0619 - mae: 0.2081 - val_loss: 0.0432 - val_mae: 0.1752

Epoch 00006: val_

Epoch 47/1000
2/2 - 0s - loss: 0.0139 - mae: 0.0936 - val_loss: 0.0215 - val_mae: 0.1155

Epoch 00047: val_mae did not improve from 0.11191
Epoch 48/1000
2/2 - 0s - loss: 0.0133 - mae: 0.0920 - val_loss: 0.0193 - val_mae: 0.1081

Epoch 00048: val_mae improved from 0.11191 to 0.10805, saving model to ../../saved_models_mlp/load_N.h5
Epoch 49/1000
2/2 - 0s - loss: 0.0131 - mae: 0.0904 - val_loss: 0.0195 - val_mae: 0.1088

Epoch 00049: val_mae did not improve from 0.10805
Epoch 50/1000
2/2 - 0s - loss: 0.0127 - mae: 0.0892 - val_loss: 0.0187 - val_mae: 0.1063

Epoch 00050: val_mae improved from 0.10805 to 0.10632, saving model to ../../saved_models_mlp/load_N.h5
Epoch 51/1000
2/2 - 0s - loss: 0.0120 - mae: 0.0864 - val_loss: 0.0179 - val_mae: 0.1039

Epoch 00051: val_mae improved from 0.10632 to 0.10391, saving model to ../../saved_models_mlp/load_N.h5
Epoch 52/1000
2/2 - 0s - loss: 0.0116 - mae: 0.0850 - val_loss: 0.0185 - val_mae: 0.1058

Epoch 00052: val_mae did not improve from 0.1039


Epoch 00097: val_mae did not improve from 0.07989
Epoch 98/1000
2/2 - 0s - loss: 0.0031 - mae: 0.0432 - val_loss: 0.0120 - val_mae: 0.0862

Epoch 00098: val_mae did not improve from 0.07989
Epoch 99/1000
2/2 - 0s - loss: 0.0030 - mae: 0.0425 - val_loss: 0.0111 - val_mae: 0.0819

Epoch 00099: val_mae did not improve from 0.07989
Epoch 100/1000
2/2 - 0s - loss: 0.0030 - mae: 0.0423 - val_loss: 0.0116 - val_mae: 0.0841

Epoch 00100: val_mae did not improve from 0.07989
Epoch 101/1000
2/2 - 0s - loss: 0.0029 - mae: 0.0418 - val_loss: 0.0116 - val_mae: 0.0842

Epoch 00101: val_mae did not improve from 0.07989
Epoch 102/1000
2/2 - 0s - loss: 0.0029 - mae: 0.0415 - val_loss: 0.0116 - val_mae: 0.0843

Epoch 00102: val_mae did not improve from 0.07989
Epoch 103/1000
2/2 - 0s - loss: 0.0029 - mae: 0.0411 - val_loss: 0.0115 - val_mae: 0.0839

Epoch 00103: val_mae did not improve from 0.07989
Epoch 104/1000
2/2 - 0s - loss: 0.0028 - mae: 0.0409 - val_loss: 0.0118 - val_mae: 0.0849

Epoch 00104: v

Epoch 154/1000
2/2 - 0s - loss: 0.0021 - mae: 0.0347 - val_loss: 0.0110 - val_mae: 0.0813

Epoch 00154: val_mae did not improve from 0.07629
Epoch 155/1000
2/2 - 0s - loss: 0.0018 - mae: 0.0323 - val_loss: 0.0121 - val_mae: 0.0866

Epoch 00155: val_mae did not improve from 0.07629
Epoch 156/1000
2/2 - 0s - loss: 0.0019 - mae: 0.0327 - val_loss: 0.0121 - val_mae: 0.0863

Epoch 00156: val_mae did not improve from 0.07629
Epoch 157/1000
2/2 - 0s - loss: 0.0018 - mae: 0.0322 - val_loss: 0.0117 - val_mae: 0.0845

Epoch 00157: val_mae did not improve from 0.07629
Epoch 158/1000
2/2 - 0s - loss: 0.0018 - mae: 0.0317 - val_loss: 0.0114 - val_mae: 0.0834

Epoch 00158: val_mae did not improve from 0.07629
Epoch 159/1000
2/2 - 0s - loss: 0.0018 - mae: 0.0315 - val_loss: 0.0110 - val_mae: 0.0818

Epoch 00159: val_mae did not improve from 0.07629
Epoch 160/1000
2/2 - 0s - loss: 0.0018 - mae: 0.0315 - val_loss: 0.0111 - val_mae: 0.0819

Epoch 00160: val_mae did not improve from 0.07629
Epoch 161/100


Epoch 00211: val_mae improved from 0.07456 to 0.07285, saving model to ../../saved_models_mlp/load_N.h5
Epoch 212/1000
2/2 - 0s - loss: 0.0016 - mae: 0.0309 - val_loss: 0.0104 - val_mae: 0.0771

Epoch 00212: val_mae did not improve from 0.07285
Epoch 213/1000
2/2 - 0s - loss: 0.0013 - mae: 0.0273 - val_loss: 0.0127 - val_mae: 0.0879

Epoch 00213: val_mae did not improve from 0.07285
Epoch 214/1000
2/2 - 0s - loss: 0.0014 - mae: 0.0285 - val_loss: 0.0109 - val_mae: 0.0797

Epoch 00214: val_mae did not improve from 0.07285
Epoch 215/1000
2/2 - 0s - loss: 0.0013 - mae: 0.0272 - val_loss: 0.0101 - val_mae: 0.0755

Epoch 00215: val_mae did not improve from 0.07285
Epoch 216/1000
2/2 - 0s - loss: 0.0013 - mae: 0.0278 - val_loss: 0.0117 - val_mae: 0.0830

Epoch 00216: val_mae did not improve from 0.07285
Epoch 217/1000
2/2 - 0s - loss: 0.0013 - mae: 0.0278 - val_loss: 0.0126 - val_mae: 0.0873

Epoch 00217: val_mae did not improve from 0.07285
Epoch 218/1000
2/2 - 0s - loss: 0.0013 - mae: 0.0


Epoch 00269: val_mae did not improve from 0.07285
Epoch 270/1000
2/2 - 0s - loss: 0.0011 - mae: 0.0245 - val_loss: 0.0106 - val_mae: 0.0766

Epoch 00270: val_mae did not improve from 0.07285
Epoch 271/1000
2/2 - 0s - loss: 9.6923e-04 - mae: 0.0233 - val_loss: 0.0121 - val_mae: 0.0833

Epoch 00271: val_mae did not improve from 0.07285
Epoch 272/1000
2/2 - 0s - loss: 0.0010 - mae: 0.0243 - val_loss: 0.0123 - val_mae: 0.0842

Epoch 00272: val_mae did not improve from 0.07285
Epoch 273/1000
2/2 - 0s - loss: 0.0010 - mae: 0.0238 - val_loss: 0.0113 - val_mae: 0.0797

Epoch 00273: val_mae did not improve from 0.07285
Epoch 274/1000
2/2 - 0s - loss: 9.6353e-04 - mae: 0.0232 - val_loss: 0.0103 - val_mae: 0.0750

Epoch 00274: val_mae did not improve from 0.07285
Epoch 275/1000
2/2 - 0s - loss: 0.0011 - mae: 0.0247 - val_loss: 0.0103 - val_mae: 0.0753

Epoch 00275: val_mae did not improve from 0.07285
Epoch 276/1000
2/2 - 0s - loss: 9.7747e-04 - mae: 0.0235 - val_loss: 0.0114 - val_mae: 0.0803

